In [4]:
import numpy as np
import cv2

class ImageClass:
    def __init__(self, image):
        self.image = image
        self.image_out = None
        self.contourCenterX = 0
        self.MainContour = None
        # self.n_slices = 10
        self.n_slices = 2
        self.detected_points = []
        self.upper_limit = 235
        self.bottom_limit = 335

        self.roi = None
        self.roi_slice = None
        self.grayscale = None
        self.binarized = None
        self.imgcontour = None


    def slice_roi(self, roi, n_slices):
        height, width = roi.shape[:2]
        slice_height = height // n_slices
        slices = []
        for i in range(n_slices):
            start_row = i * slice_height
            end_row = (i + 1) * slice_height if i < n_slices - 1 else height
            roi_slice = roi[start_row:end_row, :]
            slices.append(roi_slice)
        return slices

    def repack_image(self, slices):
        img = slices[0]
        for slice_img in slices[1:]:
            img = np.concatenate((img, slice_img), axis=0)
        img_full = np.concatenate((self.image[:self.upper_limit, :], img), axis=0)
        img_full = np.concatenate((img_full, self.image[self.bottom_limit:, :]), axis=0)
        return img_full

    def process_image(self):
        img_roi = self.image[self.upper_limit:self.bottom_limit, :]
        roi_slices = self.slice_roi(img_roi, self.n_slices)
        
        self.roi = img_roi.copy()
        self.roi_slice = roi_slices[-1]

        for i, roi_slice in enumerate(roi_slices):
            imgray = cv2.cvtColor(roi_slice, cv2.COLOR_BGR2GRAY)
            _, thresh = cv2.threshold(imgray, 100, 255, cv2.THRESH_BINARY_INV)
            # thresh = cv2.bitwise_not(thresh)  # Inverter pixels para linha preta em fundo claro
            self.grayscale = imgray
            self.binarized = thresh

            contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            self.imgcontour = cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)

            if contours:
                cv2.drawContours(self.imgcontour, contours, -1, (0, 255, 0), 2)    
                self.MainContour = self.findMainContour(contours)
                contourCenter = self.getContourCenter(self.MainContour)
                if contourCenter:
                    self.contourCenterX = contourCenter[0]
                    width = roi_slice.shape[1]
                    center_offset = contourCenter[0] - width // 2
                    self.detected_points.append(center_offset)

                    # Converter as coordenadas do centro do contorno para a imagem completa
                    global_center = (contourCenter[0], contourCenter[1] + self.upper_limit + i * (roi_slice.shape[0]))
                    
                    # Desenhar o contorno e o ponto azul na imagem original
                    cv2.drawContours(self.image, [self.MainContour + np.array([[0, self.upper_limit + i * (roi_slice.shape[0])]])], -1, (0, 255, 0), 2)
                    # cv2.circle(self.image, global_center, 3, (255, 0, 0), -1)
                    
                    # Adicionar o texto das coordenadas do ponto na imagem original
                    # text_position = (global_center[0] + 5, global_center[1] - 5)
                    # cv2.putText(self.image, f"({global_center[0]}, {global_center[1]})", 
                    #             text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)

            # Ponto branco de referência no centro do slice na imagem original
            # slice_height = roi_slice.shape[0]
            # slice_center = (width // 2, slice_height // 2 + self.upper_limit + i * slice_height)
            # cv2.circle(self.image, slice_center, 3, (255, 255, 255), -1)

        self.image_out = self.image
        final_height, final_width = self.image_out.shape[:2]
        cv2.line(self.image_out, (0, self.bottom_limit), (final_width, self.bottom_limit), (0, 0, 255), 2)
        cv2.line(self.image_out, (0, self.upper_limit), (final_width, self.upper_limit), (0, 0, 255), 2)

    def findMainContour(self, contours):
        biggestContour = max(contours, key=cv2.contourArea)

        # Verifique se o contorno está abaixo da linha preta
        contourCenter = self.getContourCenter(biggestContour)
        if contourCenter and contourCenter[1] > self.bottom_limit:  # Certifique-se de que Y está abaixo da linha
            return biggestContour

        if self.detected_points:
            if self.getContourCenter(biggestContour):
                biggestContourX = self.getContourCenter(biggestContour)[0]
                if abs((self.contourCenterX - biggestContourX) - self.detected_points[-1]) > 50:
                    closest_contour = biggestContour
                    closest_contourX = biggestContourX
                    for contour in contours:
                        contourCenter = self.getContourCenter(contour)
                        if contourCenter and contourCenter[1] > self.bottom_limit:  # Verificar a linha
                            temp_contourX = contourCenter[0]
                            if abs((self.contourCenterX - temp_contourX) - self.detected_points[-1]) < \
                               abs((self.contourCenterX - closest_contourX) - self.detected_points[-1]):
                                closest_contour = contour
                                closest_contourX = temp_contourX
                    return closest_contour
                else:
                    return biggestContour
        return biggestContour

    def getContourCenter(self, contour):
        M = cv2.moments(contour)
        if M["m00"] == 0:
            return None
        x = int(M["m10"] / M["m00"])
        y = int(M["m01"] / M["m00"])
        return [x, y]

def detect_lane_image(image):
    processImage = ImageClass(image)
    processImage.process_image()
    return processImage.image_out, processImage.detected_points

                            
                            
# Read image
img = cv2.imread('assets/04/frame0000.jpg')
processImage = ImageClass(img)
processImage.process_image()
cv2.imwrite('assets/04/out/out.png', processImage.image_out)
cv2.imwrite('assets/04/out/roi.png', processImage.roi)
cv2.imwrite('assets/04/out/roi_slice.png', processImage.roi_slice)
cv2.imwrite('assets/04/out/grayscale.png', processImage.grayscale)
cv2.imwrite('assets/04/out/binarized.png', processImage.binarized)
cv2.imwrite('assets/04/out/imgcontour.png', processImage.imgcontour)
print('detected pointss: ', processImage.detected_points)             

AttributeError: 'ImageClass' object has no attribute 'roi'